In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from nltk.metrics import distance as distance
from sklearn.model_selection import StratifiedKFold
from MeanEncoder import MeanEncoder

In [3]:
train = pd.read_json("train.json")
test = pd.read_json("test.json")

In [4]:
y_map = {'low': 2, 'medium': 1, 'high': 0}
train['interest_level'] = train['interest_level'].apply(lambda x: y_map[x])

y_train = train.interest_level
train = train.drop(['listing_id', 'interest_level'], axis=1)

listing_id = test.listing_id.values
test = test.drop('listing_id', axis=1)

ntrain = train.shape[0]
train_test = pd.concat((train, test), axis=0).reset_index(drop=True)

In [60]:
train

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],40.7145,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",40.7947,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",40.7388,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",40.7539,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],40.8241,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street
100014,2.0,4,38a913e46c94a7f46ddf19b756a9640c,2016-04-19 04:24:47,,West 18th Street,[],40.7429,-74.0028,b209e2c4384a64cc307c26759ee0c651,[https://photos.renthop.com/2/6894514_9abb8592...,7995,350 West 18th Street
100016,1.0,2,3ba49a93260ca5df92fde024cb4ca61f,2016-04-27 03:19:56,Stunning unit with a great location and lots o...,West 107th Street,"[prewar, elevator, Dogs Allowed, Cats Allowed,...",40.8012,-73.9660,01287194f20de51872e81f660def4784,[https://photos.renthop.com/2/6930771_7e3622b6...,3600,210 West 107th Street
100020,2.0,1,0372927bcb6a0949613ef5bf893bbac7,2016-04-13 06:01:42,"This huge sunny ,plenty of lights 1 bed/2 bath...",West 21st Street,"[Doorman, Elevator, Pre-War, Terrace, Laundry ...",40.7427,-73.9957,e6472c7237327dd3903b3d6f6a94515a,[https://photos.renthop.com/2/6867392_b18283f6...,5645,155 West 21st Street
100026,1.0,1,a7efbeb58190aa267b4a9121cd0c88c0,2016-04-20 02:36:35,<p><a website_redacted,Hamilton Terrace,"[Cats Allowed, Dogs Allowed, Elevator, Laundry...",40.8234,-73.9457,c1a6598437b7db560cde66e5a297a53f,[https://photos.renthop.com/2/6898799_3759be4c...,1725,63 Hamilton Terrace
100027,2.0,4,0,2016-04-02 02:58:15,This is a spacious four bedroom with every bed...,522 E 11th,"[Dishwasher, Hardwood Floors]",40.7278,-73.9808,23a01ea7717b38875f5b070282d1b9d2,[https://photos.renthop.com/2/6814332_e19a8552...,5800,522 E 11th


In [6]:
#remove some noise
#ulimit = np.percentile(train_test.price.values, 99)
train_test['price'].ix[train_test['price']>13000] = 13000

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
# remove some noise
train_test.loc[train_test["bathrooms"] == 112, "bathrooms"] = 1.5
train_test.loc[train_test["bathrooms"] == 10, "bathrooms"] = 1
train_test.loc[train_test["bathrooms"] == 20, "bathrooms"] = 2

In [8]:
train_test['price_bathrooms'] =  (train_test["price"])/ (train_test["bathrooms"] +1.0)
train_test['price_bedrooms'] =  (train_test["price"])/ (train_test["bedrooms"] +1.0)

In [9]:
train_test["room_diff"] = train_test["bathrooms"] - train_test["bedrooms"]
train_test["room_num"] = train_test["bedrooms"] + train_test["bathrooms"]

In [10]:
train_test['Date'] = pd.to_datetime(train_test['created'])
train_test['Year'] = train_test['Date'].dt.year
train_test['Month'] = train_test['Date'].dt.month
train_test['Day'] = train_test['Date'].dt.day
train_test['Wday'] = train_test['Date'].dt.dayofweek
train_test['Yday'] = train_test['Date'].dt.dayofyear
train_test['hour'] = train_test['Date'].dt.hour

train_test = train_test.drop(['Date', 'created'], axis=1)

In [11]:
# count of words present in description column #
train_test["num_description_words"] = train_test["description"].apply(lambda x: len(x.split(" ")))
train_test = train_test.drop(['description'], axis=1)

In [12]:
managers_count = train_test['manager_id'].value_counts()

train_test['top_10_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
    managers_count.values >= np.percentile(managers_count.values, 90)] else 0)
train_test['top_25_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
    managers_count.values >= np.percentile(managers_count.values, 75)] else 0)
train_test['top_5_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
    managers_count.values >= np.percentile(managers_count.values, 95)] else 0)
train_test['top_50_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
    managers_count.values >= np.percentile(managers_count.values, 50)] else 0)
train_test['top_1_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
    managers_count.values >= np.percentile(managers_count.values, 99)] else 0)
train_test['top_2_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
    managers_count.values >= np.percentile(managers_count.values, 98)] else 0)
train_test['top_15_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
    managers_count.values >= np.percentile(managers_count.values, 85)] else 0)
train_test['top_20_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
    managers_count.values >= np.percentile(managers_count.values, 80)] else 0)
train_test['top_30_manager'] = train_test['manager_id'].apply(lambda x: 1 if x in managers_count.index.values[
    managers_count.values >= np.percentile(managers_count.values, 70)] else 0)

In [13]:
buildings_count = train_test['building_id'].value_counts()

train_test['top_10_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
    buildings_count.values >= np.percentile(buildings_count.values, 90)] else 0)
train_test['top_25_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
    buildings_count.values >= np.percentile(buildings_count.values, 75)] else 0)
train_test['top_5_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
    buildings_count.values >= np.percentile(buildings_count.values, 95)] else 0)
train_test['top_50_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
    buildings_count.values >= np.percentile(buildings_count.values, 50)] else 0)
train_test['top_1_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
    buildings_count.values >= np.percentile(buildings_count.values, 99)] else 0)
train_test['top_2_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
    buildings_count.values >= np.percentile(buildings_count.values, 98)] else 0)
train_test['top_15_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
    buildings_count.values >= np.percentile(buildings_count.values, 85)] else 0)
train_test['top_20_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
    buildings_count.values >= np.percentile(buildings_count.values, 80)] else 0)
train_test['top_30_building'] = train_test['building_id'].apply(lambda x: 1 if x in buildings_count.index.values[
    buildings_count.values >= np.percentile(buildings_count.values, 70)] else 0)

In [14]:
 # Clustering
train_location = train_test.loc[:ntrain-1, ['latitude', 'longitude']]
test_location = train_test.loc[ntrain:, ['latitude', 'longitude']]

kmeans_cluster = KMeans(n_clusters=20)
res = kmeans_cluster.fit(train_location)
res = kmeans_cluster.predict( pd.concat((train_location, test_location), axis=0).reset_index(drop=True))

train_test['cenroid'] = res

# L1 distance
center = [ train_location['latitude'].mean(), train_location['longitude'].mean()]
train_test['distance'] = abs(train_test['latitude'] - center[0]) + abs(train_test['longitude'] - center[1])

In [15]:
train_test['display_address'] = train_test['display_address'].apply(lambda x: x.lower().strip())

In [16]:
train_test['street_address'] = train_test['street_address'].apply(lambda x: x.lower().strip())

In [17]:
categoricals = ['building_id', 'manager_id', 'display_address', 'street_address']
#categoricals = [x for x in train_test.columns if train_test[x].dtype == 'object']
for feat in categoricals:
    lbl = LabelEncoder()
    lbl.fit(list(train_test[feat].values))
    train_test[feat] = lbl.transform(list(train_test[feat].values))

In [18]:
me = MeanEncoder(categoricals)

#trian
#import pdb
#pdb.set_trace()
train_new = train_test.iloc[:ntrain, :]
train_new_cat = me.fit_transform(train_new, y_train)

#test
test_new = train_test.iloc[ntrain:, :]
test_new_cat = me.transform(test_new)

In [19]:
train_test = pd.concat((train_new_cat, test_new_cat), axis=0).reset_index(drop=True)
train_test.drop(categoricals, axis=1, inplace=True)

In [20]:
train_test['features_count'] = train_test['features'].apply(lambda x: len(x))
train_test['features2'] = train_test['features']
train_test['features2'] = train_test['features2'].apply(lambda x: ' '.join(x))

c_vect = CountVectorizer(stop_words='english', max_features=200, ngram_range=(1, 1))
c_vect_sparse = c_vect.fit_transform(train_test['features2'])
c_vect_sparse_cols = c_vect.get_feature_names()

train_test.drop(['features', 'features2'], axis=1, inplace=True)


In [21]:
train_test['photos_count'] = train_test['photos'].apply(lambda x: len(x))
train_test.drop(['photos'], axis=1, inplace=True)

In [47]:
train_test

,bathrooms,bedrooms,latitude,longitude,price,price_bathrooms,price_bedrooms,room_diff,room_num,Year,...,manager_id_pred_1,manager_id_pred_2,display_address_pred_0,display_address_pred_1,display_address_pred_2,street_address_pred_0,street_address_pred_1,street_address_pred_2,features_count,photos_count
0,1.5,3,40.7145,-73.9425,3000,1200.000000,750.000000,-1.5,4.5,2016,...,0.078591,0.334445,0.009369,0.026352,0.270365,0.009369,0.026352,0.083698,0,5
1,1.0,2,40.7947,-73.9667,5465,2732.500000,1821.666667,-1.0,3.0,2016,...,0.061352,0.276198,0.023753,0.112653,0.337808,0.009369,0.026352,0.521198,5,11
2,1.0,1,40.7388,-74.0018,2850,1425.000000,1425.000000,0.0,2.0,2016,...,0.085299,0.245804,0.022577,0.118805,0.281812,0.039804,0.148091,0.296742,4,8
3,1.0,1,40.7539,-73.9677,3275,1637.500000,1637.500000,0.0,2.0,2016,...,0.085674,0.255732,0.032702,0.073019,0.247032,0.051369,0.068352,0.237698,2,3
4,1.0,4,40.8241,-73.9493,3350,1675.000000,670.000000,-3.0,5.0,2016,...,0.026352,0.338244,0.009369,0.026352,0.258698,0.009369,0.026352,0.083698,1,3
5,2.0,4,40.7429,-74.0028,7995,2665.000000,1599.000000,-2.0,6.0,2016,...,0.086108,0.288577,0.038536,0.099269,0.258698,0.009369,0.126352,0.283698,0,5
6,1.0,2,40.8012,-73.9660,3600,1800.000000,1200.000000,-1.0,3.0,2016,...,0.026352,0.550365,0.016737,0.048457,0.326856,0.009369,0.026352,0.258698,8,10
7,2.0,1,40.7427,-73.9957,5645,1881.666667,2822.500000,1.0,3.0,2016,...,0.091241,0.281990,0.048992,0.092390,0.228982,0.009369,0.026352,0.317032,8,5
8,1.0,1,40.8234,-73.9457,1725,862.500000,862.500000,0.0,2.0,2016,...,0.125070,0.299083,0.009369,0.063194,0.194225,0.048258,0.026352,0.200365,4,5
9,2.0,4,40.7278,-73.9808,5800,1933.333333,1160.000000,-2.0,6.0,2016,...,0.113852,0.258698,0.031230,0.087840,0.278995,0.031230,0.087840,0.278995,2,9


In [24]:
#存为csv格式方便用excel查看
X_train = train_test.iloc[:ntrain, :]
X_test = train_test.iloc[ntrain:, :]


In [33]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(49352, 50)
(74659, 50)
(49352,)


In [44]:
y_train

10        1
10000     2
100004    0
100007    2
100013    2
100014    1
100016    2
100020    2
100026    1
100027    2
100030    2
10004     2
100044    0
100048    2
10005     2
100051    1
100052    2
100053    2
100055    2
100058    2
100062    2
100063    1
100065    2
100066    2
10007     1
100071    2
100075    1
100076    2
100079    0
100081    2
         ..
99915     2
99917     2
99919     1
99921     1
99923     2
99924     2
99931     2
99933     2
99935     2
99937     2
9994      2
99953     2
99956     2
99960     1
99961     2
99964     1
99965     2
99966     2
99979     2
99980     2
99982     0
99984     2
99986     2
99987     2
99988     1
9999      1
99991     2
99992     2
99993     2
99994     2
Name: interest_level, dtype: int64

In [57]:
train_new = pd.concat((X_train.reset_index(), y_train.reset_index()), axis=1)
train_new = train_new.drop(['index'], axis = 1)

In [61]:
X_test

,bathrooms,bedrooms,latitude,longitude,price,price_bathrooms,price_bedrooms,room_diff,room_num,Year,...,manager_id_pred_1,manager_id_pred_2,display_address_pred_0,display_address_pred_1,display_address_pred_2,street_address_pred_0,street_address_pred_1,street_address_pred_2,features_count,photos_count
49352,1.0,1,40.7185,-73.9865,2950,1475.000000,1475.000000,0.0,2.0,2016,...,0.111934,0.348179,0.009191,0.026601,0.316679,0.009191,0.026601,0.083346,6,8
49353,1.0,2,40.7278,-74.0000,2850,1425.000000,950.000000,-1.0,3.0,2016,...,0.110990,0.083346,0.031317,0.080266,0.270430,0.009191,0.123823,0.248624,3,3
49354,1.0,1,40.7306,-73.9890,3758,1879.000000,1879.000000,0.0,2.0,2016,...,0.038962,0.122214,0.013492,0.038962,0.122214,0.013492,0.225629,0.122214,3,6
49355,1.0,2,40.7109,-73.9571,3300,1650.000000,1100.000000,-1.0,3.0,2016,...,0.152690,0.267443,0.030637,0.088669,0.277821,0.030637,0.088669,0.277821,10,6
49356,2.0,2,40.7650,-73.9845,4900,1633.333333,1633.333333,0.0,4.0,2016,...,0.085030,0.248714,0.009191,0.026601,0.083346,0.029741,0.077900,0.309456,14,7
49357,3.0,3,40.7901,-73.9774,9000,2250.000000,2250.000000,0.0,6.0,2016,...,0.097417,0.264173,0.066386,0.084573,0.255068,0.242524,0.026601,0.083346,11,8
49358,1.0,2,40.7730,-73.9571,2800,1400.000000,933.333333,-1.0,3.0,2016,...,0.085618,0.279088,0.009191,0.110789,0.272403,0.009191,0.026601,0.293346,2,8
49359,1.0,0,40.6751,-73.9511,1900,950.000000,1900.000000,1.0,1.0,2016,...,0.026601,0.289346,0.219191,0.236601,0.083346,0.219191,0.236601,0.083346,5,5
49360,1.0,2,40.7597,-73.9929,3000,1500.000000,1000.000000,-1.0,3.0,2016,...,0.091004,0.274908,0.014115,0.085973,0.302318,0.009191,0.516601,0.083346,2,3
49361,1.0,0,40.7208,-73.9887,2300,1150.000000,2300.000000,1.0,1.0,2016,...,0.048534,0.225118,0.047222,0.064148,0.253018,0.359191,0.026601,0.083346,2,3


In [62]:
train_new.to_csv( 'RentListingInquries_FE_train.csv', index=False)
X_test.to_csv( 'RentListingInquries_FE_test.csv', index=False)